# Adiabatic Pipe Flow

## Introduction

Consider the following information on a well insulated steam pipe from measurement equipment:

| Parameter | Location | Value | Unit |
|:--------- |:-------- | -----:| ---- |
| Temperature | Inlet | 220 | °C |
| Pressure | Inlet | 10 | bar |
|  | Outlet | 6 | bar |

With the energy balance of the thermodynamic open system, where the work $\dot W$ and heat $\dot Q$ transferred change the enthalpy $h$ of a mass flow $\dot m$ from a state 1 to a different state 2, we can derive the missing outlet temperature:

```{math}
    :label: eq:pipe-flow-energy-balance
    \dot W + \dot Q = \dot m \cdot \left(h_2 - h_1\right)
```

Since a pipe does not transfer work to the fluid and assuming it is a well insulated pipe, work and heat transferred can be considered equal to zero. Therefore the change in enthalpy must be zero as well:

```{math}
    :label: eq:pipe-flow-energy-balance-simplified
    0 = \dot m \cdot \left(h_2 - h_1\right)\\
    h_2 = h_1
```

We cam then also obtain the outlet temperature, as it can be calculated based on the pressure and the enthalpy:

```{math}
    :label: eq:pipe-flow-outlet-temperature
    T_2 = T\left(p_2, h_1\right)\\
    h_1 = \left(T_1, p_1\right)
```

To do that computationally, we first import the `PropsSI` function from `CoolProp` and then insert the values from the table above.

In [ ]:
from CoolProp.CoolProp import PropsSI as PSI


fluid = "water"

p_in = 10 * 1e5
T_in = 195 + 273.15
h_in = PSI("H", "P", p_in, "T", T_in, fluid)

p_out = 6 * 1e5
h_out = h_in
T_out = PSI("T", "P", p_out, "H", h_out, fluid)
T_out_celsius = T_out - 273.15

T_out_celsius

## Exergy Analysis

No energy has been transferred in the pipe flow. However, the process is not reversible: You cannot change the pressure of the fluid at the outlet back to the inlet pressure. That means, while we have not lost any energy to the ambient, the energy must have become less valuable. We can make this change in quality visible using Second Law analysis. To do that, we calculate the exergy of the fluid at the inlet and at the outlet of the pipe. First, we define a function, that follows the definition of phyiscal exergy in eq. {eq}`physical-exergy`. Chemical exergy can be ignored in this application, since no chemical reaction processes take place.

The function `calc_physical_exergy` will take pressure and temperature of a fluid and calculate the thermal and the mechanical part of the physical exergy.

In [ ]:
def calc_physical_exergy(p, T, p0, T0, fluid):
    r"""
    Calculate specific physical exergy.

    Physical exergy is allocated to a thermal and a mechanical share.

    Parameters
    ----------
    p : float
        Pressure of fluid p / Pa.
        
    T : float
        Temperature of fluid T / K.

    p0 : float
        Ambient pressure p0 / Pa.

    T0 : float
        Ambient temperature T0 / K.

    Returns
    -------
    e_ph : tuple
        Specific thermal and mechanical exergy
        (:math:`e^\mathrm{T}`, :math:`e^\mathrm{M}`) in J / kg.
    """
    h = PSI("H", "P", p, "T", T, fluid)
    s = PSI("S", "P", p, "T", T, fluid)
    
    h_T0_p = PSI("H", "P", p, "T", T0, fluid)
    s_T0_p = PSI("S", "P", p, "T", T0, fluid)
    
    ex_therm = (h - h_T0_p) - T0 * (s - s_T0_p)
    
    h0 = PSI("H", "P", p0, "T", T0, fluid)
    s0 = PSI("S", "P", p0, "T", T0, fluid)
    
    ex_mech = (h_T0_p - h0) - T0 * (s_T0_p - s0)
    return ex_therm, ex_mech

Then, we can define an (arbitrary) ambient state and calculate the exergy and the inlet and the outlet state.

In [ ]:
p0 = 1.01325 * 1e5
T0 = 20 + 273.15


ex_in = calc_physical_exergy(p_in, T_in, p0, T0, fluid)
ex_in

In [ ]:
ex_out = calc_physical_exergy(p_out, T_out, p0, T0, fluid)
ex_out

First, we can observe the change in total specific exergy or the change of exergy in relation to the exergy available at the pipe's inlet:

In [ ]:
exergy_destruction_total = sum(ex_out) - sum(ex_in)
exergy_destruction_total

In [ ]:
from myst_nb import glue

In [ ]:
exergy_destruction_ratio = (sum(ex_in) - sum(ex_out)) / sum(ex_in)
exergy_destruction_ratio

In [ ]:
glue("exergy_destruction_total", exergy_destruction_total, display=False)
glue("exergy_destruction_ratio", exergy_destruction_ratio * 100, display=False)

We see that a total of {glue:text}`exergy_destruction_total:.0f` J/kg is destroyed, which corresponds to about {glue:text}`exergy_destruction_ratio:.2f` % of input exergy.

## Excercise

With the setup demonstrated in the above section, consider the following excercises:

1. Calculate the total exergy destruction and the destruction share of the pipe flow for varying outlet pressures in a range from 1 to 10 bars.
2. Plot the results to show how the exergy destruction (y-axis) depends on the pressure ratio (x-axis).
3. Analyse the thermal and the mechanical shares of exergy. Plot these as well.
4. Why does the thermal exergy not change with pressure values below 3 bars.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

pressure_range = np.linspace(p0 / 1e5, 10) * 1e5
outlet_temperature_range = PSI("T", "P", pressure_range, "H", h_in, fluid)

ex_T, ex_M = calc_physical_exergy(pressure_range, outlet_temperature_range, p0, T0, fluid)

exergy_destruction_total = sum(ex_in) - (ex_M + ex_T)
exergy_destruction_ratio = exergy_destruction_total / sum(ex_in)
exergy_destruction_ratio

# plt.plot(pressure_range, exergy_destruction_ratio)


# plt.plot(pressure_range, ex_M + ex_T)


# plt.plot(pressure_range, ex_M / (ex_M + ex_T))
# plt.plot(pressure_range, ex_T / ex_in[1])
# plt.plot(pressure_range, ex_M / ex_in[0])

pressure_range / p_in
plt.plot(pressure_range / 1e5, ex_M / ex_in[1])
plt.plot(pressure_range / 1e5, ex_T / ex_in[0])
# plt.plot(pressure_range, )

ex_M

